In [56]:
import warnings


warnings.filterwarnings("ignore")

import nltk
from nltk import FreqDist
nltk.download('stopwords')

# !pip install pyLDAvis

import pandas as pd
# pd.set_option("display.max_colwidth", 200)
import numpy as np
import json
import re
import gzip
import spacy

import gensim
from gensim import corpora

import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from gensim.models.coherencemodel import CoherenceModel

from nltk.corpus import stopwords

import nltk

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

from nltk.stem import WordNetLemmatizer

from tqdm import tqdm

import gensim
from gensim import corpora

import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tanma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [57]:
import json

In [58]:
from gensim.test.utils import datapath

## Loading in the preprocessed data

In [59]:
engaged_df = pd.read_csv("./data/engaged_df.csv")

In [60]:
engaged_df.head()

,book_id,user_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,clean_reviews
0,5805,4cbecbc15af3db041a8e0f594c642bb5,58f2301bd2d4bbfc1b51e4e5fb161cfe,5,"Remember, remember, the fifth of November. Thi...",Wed Jun 13 17:55:53 -0700 2012,Mon Jan 30 05:58:01 -0800 2017,Thu Jun 14 00:00:00 -0700 2012,NaN,7,0,"['remember', 'remember', 'fifth', 'part', 'quo..."
1,5805,49cc59f1c479d698507627b401d47ecf,761a17f52538341a085b629a316204a1,4,Tinha apontado este livro como um dos que tinh...,Sun Jan 05 03:27:10 -0800 2014,Tue Oct 28 15:58:20 -0700 2014,Tue Oct 28 15:58:20 -0700 2014,NaN,1,0,"['tinha', 'visto', 'portman', 'hora', 'livro',..."
2,5805,5f03864c758bfceb6d7d5e93eeb20044,ac54c3ce0c9f660c03881b0668f79c60,5,Review coming soon! www.youtube.com/ReadTomes,Tue Feb 12 11:38:36 -0800 2013,Tue Feb 12 12:00:47 -0800 2013,Tue Feb 12 12:00:47 -0800 2013,Tue Feb 12 00:00:00 -0800 2013,1,0,"['review', 'ing', 'soon', 'readtome']"
3,5805,c309dff1695ed8558b29ea8dcd7479b8,0da0bcc469c2acd15350f9a8f0a74e2b,5,"What better way to celebrate Guy Fawkes Day, t...",Tue May 20 09:52:36 -0700 2014,Thu Mar 16 09:49:11 -0700 2017,Wed Nov 05 14:22:10 -0800 2014,Wed Nov 05 00:00:00 -0800 2014,2,0,"['well', 'way', 'celebrate', 'guy', 'day', 're..."
4,5805,0f6b8c04f811e05c8978bd6b66ce7685,7d670b6c8cac0c086e21ae2f1af6eccb,4,To note - I am writing this review a quarter c...,Tue Jul 03 16:43:26 -0700 2012,Fri Feb 27 14:46:00 -0800 2015,Tue Jan 01 00:00:00 -0800 1991,NaN,10,0,"['note', 'write', 'review', 'quarter', 'centur..."


In [61]:
def to_list(x):
    x = x.strip('][').split(', ')
#     print(x)
    x = [i.strip("'") for i in x]
    return x

In [62]:
engaged_df['clean_reviews'] = engaged_df['clean_reviews'].apply(lambda x: to_list(x))

## LDA Training over the reviews

In [63]:
dictionary = corpora.Dictionary(engaged_df['clean_reviews'])

In [64]:
doc_term_matrix = tqdm([dictionary.doc2bow(rev) for rev in engaged_df['clean_reviews']])

  0%|                                                                                        | 0/39787 [44:48<?, ?it/s]


In [65]:
## Setting the Hyperparameter Range

# Topics range
min_topics = 2
max_topics = 9
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')


In [66]:
model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

In [67]:
## Using the coherence metric to tune the LDA model
def compute_coherence_values(corpus, dictionary, k, a, b, test_sample):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=test_sample, dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [68]:
len(engaged_df['clean_reviews'])

39787

In [69]:
for k in tqdm(topics_range, ):
    # iterate through alpha values
    for a in tqdm(alpha):
        # iterare through beta values
        for b in tqdm(beta):
            test_sample = np.random.choice(engaged_df['clean_reviews'], size = 7500)
            cv = compute_coherence_values(corpus=doc_term_matrix, 
                                           dictionary=dictionary, 
                                           k=k, 
                                           a=a, 
                                           b=b,
                                           test_sample = test_sample)
                
            model_results['Topics'].append(k)
            model_results['Alpha'].append(a)
            model_results['Beta'].append(b)
            model_results['Coherence'].append(cv)
            
pd.DataFrame(model_results).to_csv('lda_reviews_tuning_results.csv', index=False)


  0%|                                                                                            | 0/7 [00:00<?, ?it/s]

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████| 39787/39787 [00:18<00:00, 2117.27it/s]



 20%|████████████████▌                                                                  | 1/5 [01:51<07:24, 111.02s/it]


 40%|█████████████████████████████████▏                                                 | 2/5 [03:59<06:04, 121.35s/it]


 60%|█████████████████████████████████████████████████▊                                 | 3/5 [06:05<04:07, 123.56s/it]


 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [08:07<02:02, 122.81s/it]


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [10:19<00:00, 123.89s/it]


100%|██████████████████

 17%|█████████████▊                                                                     | 1/6 [10:16<51:21, 616.21s/it]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▌                                                                  | 1/5 [02:04<08:19, 124.81s/it]

 40%|█████████████████████████████████▏                                                 | 2/5 [04:08<06:13, 124.41s/it]

 60%|█████████████████████████████████████████████████▊                                 | 3/5 [06:14<04:09, 124.74s/it]

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [08:18<02:04, 124.81s/it]

 33%|███████████████████████████▋                                                       | 2/6 [20:38<41:18, 619.72s/it]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▌          

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▌                                                                  | 1/5 [02:09<08:38, 129.55s/it]

 40%|█████████████████████████████████▏                                                 | 2/5 [04:22<06:34, 131.48s/it]

 60%|█████████████████████████████████████████████████▊                                 | 3/5 [06:30<04:19, 129.93s/it]

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [08:36<02:08, 128.46s/it]

 83%|█████████████████████████████████████████████████████████████████████▏             | 5/6 [53:22<10:41, 641.28s/it]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▌                                                                  | 1/5 [02:08<08:33, 128.43s/it]

 40%|███████████████████████████

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▌                                                                  | 1/5 [02:17<09:11, 137.94s/it]

 40%|█████████████████████████████████▏                                                 | 2/5 [04:35<06:52, 137.51s/it]

 60%|█████████████████████████████████████████████████▊                                 | 3/5 [06:52<04:34, 137.39s/it]

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [09:08<02:16, 136.96s/it]

 33%|███████████████████████████▋                                                       | 2/6 [22:48<45:37, 684.28s/it]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▌                                                                  | 1/5 [02:18<09:12, 138.18s/it]

 40%|███████████████████████████

 20%|████████████████▌                                                                  | 1/5 [02:24<09:36, 144.10s/it]

 40%|█████████████████████████████████▏                                                 | 2/5 [04:46<07:08, 142.90s/it]

 60%|█████████████████████████████████████████████████▊                                 | 3/5 [07:10<04:47, 143.78s/it]

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [09:32<02:22, 142.75s/it]

 83%|█████████████████████████████████████████████████████████████████████▏             | 5/6 [59:21<11:52, 712.72s/it]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▌                                                                  | 1/5 [02:21<09:27, 141.76s/it]

 40%|█████████████████████████████████▏                                                 | 2/5 [04:44<07:07, 142.39s/it]

 60%|███████████████████████████

## Train and Save Best Model

In [70]:
res = pd.DataFrame(model_results)

In [73]:
res.sort_values(by = "Coherence", ascending = False)

,Topics,Alpha,Beta,Coherence
118,5,asymmetric,0.91,0.405166
193,8,0.61,0.91,0.401000
189,8,0.31,symmetric,0.392481
177,7,asymmetric,0.61,0.388753
33,3,0.01,0.91,0.387699
...,...,...,...,...
183,8,0.01,0.91,NaN
188,8,0.31,0.91,NaN
202,8,symmetric,0.61,NaN
203,8,symmetric,0.91,NaN


In [74]:
lda_model = gensim.models.LdaMulticore(corpus=doc_term_matrix,
                                           id2word=dictionary,
                                           num_topics=5, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha="asymmetric",
                                           eta=0.91)

In [79]:
save_path = datapath('./data/best_review_model')


In [80]:
save_path

'C:\\Users\\tanma\\Anaconda3\\lib\\site-packages\\gensim\\test\\test_data\\./data/best_review_model'

In [81]:
lda_model.save('./data/best_review_model')

## LDA Training Over The Descriptions

In [82]:
book_df = pd.read_csv('./data/book_p.csv')

In [83]:
book_df['processed_description'] = book_df['processed_description'].apply(lambda x: to_list(x))

In [84]:
book_df.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series,processed_description
0,1401207928,3400,['631559'],US,eng,"[{'count': '42534', 'name': 'to-read'}, {'coun...",NaN,False,4.25,B0064W65UM,...,NaN,2005.0,https://www.goodreads.com/book/show/5805.V_for...,https://images.gr-assets.com/books/1343668985m...,5805,212464,392838,V for Vendetta,V for Vendetta,"[remember, remember, fifth, frightening, power..."
1,394541553,4402,['186595'],US,eng,"[{'count': '92915', 'name': 'to-read'}, {'coun...",NaN,False,4.35,NaN,...,NaN,1991.0,https://www.goodreads.com/book/show/15196.Maus_I,https://images.gr-assets.com/books/1327884972m...,15196,189288,1947012,Maus I: A Survivor's Tale: My Father Bleeds Hi...,Maus I: A Survivor's Tale: My Father Bleeds Hi...,"[story, jewish, survivor, son, cartoonist, try..."
2,1401207529,1790,"['482441', '300099', '194163', '636672']",US,eng,"[{'count': '7924', 'name': 'to-read'}, {'count...",NaN,False,4.22,B0064W65SO,...,NaN,2005.0,https://www.goodreads.com/book/show/59980.Batman,https://images.gr-assets.com/books/1327940389m...,59980,154013,2501570,Batman: Year One,Batman: Year One,"[lieutenant, take, new, post, crime, ride, cor..."
3,1421501686,2399,['205782'],US,eng,"[{'count': '9727', 'name': 'to-read'}, {'count...",NaN,False,4.42,NaN,...,NaN,2005.0,https://www.goodreads.com/book/show/13615.Deat...,https://images.gr-assets.com/books/1419952134m...,13615,142755,1782155,"Death Note, Vol. 1: Boredom (Death Note, #1)","Death Note, Vol. 1: Boredom (Death Note, #1)","[light, ace, student, great, prospect, bore, m..."
4,1607066017,8700,['736247'],US,eng,"[{'count': '78762', 'name': 'to-read'}, {'coun...",NaN,False,4.24,B015XEABR4,...,NaN,2012.0,https://www.goodreads.com/book/show/15704307-s...,https://images.gr-assets.com/books/1486028947m...,15704307,142640,19113524,"Saga, Vol. 1 (Saga, #1)","Saga, Vol. 1 (Saga, #1)","[two, soldier, opposite, side, never, end, gal..."


In [85]:
dictionary = corpora.Dictionary(book_df['processed_description'])

In [86]:
doc_term_matrix = tqdm([dictionary.doc2bow(rev) for rev in book_df['processed_description']])

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

In [87]:
## Setting the Hyperparameter Range

# Topics range
min_topics = 2
max_topics = 9
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')


In [88]:
model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

In [89]:
for k in tqdm(topics_range, ):
    # iterate through alpha values
    for a in tqdm(alpha):
        # iterare through beta values
        for b in tqdm(beta):
            test_sample = np.random.choice(book_df['processed_description'], size = 7500)
            cv = compute_coherence_values(corpus=doc_term_matrix, 
                                           dictionary=dictionary, 
                                           k=k, 
                                           a=a, 
                                           b=b,
                                           test_sample = test_sample)
                
            model_results['Topics'].append(k)
            model_results['Alpha'].append(a)
            model_results['Beta'].append(b)
            model_results['Coherence'].append(cv)
            
pd.DataFrame(model_results).to_csv('lda_description_tuning_results.csv', index=False)


  0%|                                                                                            | 0/7 [00:00<?, ?it/s]

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:06<00:00, 161.52it/s]



 20%|████████████████▊                                                                   | 1/5 [00:12<00:49, 12.36s/it]


 40%|█████████████████████████████████▌                                                  | 2/5 [00:24<00:37, 12.38s/it]


 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:37<00:24, 12.42s/it]


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:49<00:12, 12.35s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:01<00:00, 12.29s/it]


100%|██████████████████

 17%|██████████████                                                                      | 1/6 [01:03<05:17, 63.52s/it]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▊                                                                   | 1/5 [00:13<00:52, 13.24s/it]

 40%|█████████████████████████████████▌                                                  | 2/5 [00:26<00:39, 13.01s/it]

 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:39<00:26, 13.04s/it]

 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:52<00:13, 13.08s/it]

 33%|████████████████████████████                                                        | 2/6 [02:08<04:17, 64.44s/it]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▊          

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▊                                                                   | 1/5 [00:12<00:50, 12.61s/it]

 40%|█████████████████████████████████▌                                                  | 2/5 [00:25<00:37, 12.62s/it]

 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:37<00:25, 12.68s/it]

 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:50<00:12, 12.69s/it]

 83%|██████████████████████████████████████████████████████████████████████              | 5/6 [05:29<01:05, 65.30s/it]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▊                                                                   | 1/5 [00:12<00:50, 12.61s/it]

 40%|███████████████████████████

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▊                                                                   | 1/5 [00:13<00:54, 13.51s/it]

 40%|█████████████████████████████████▌                                                  | 2/5 [00:27<00:41, 13.73s/it]

 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:41<00:27, 13.68s/it]

 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:54<00:13, 13.65s/it]

 33%|████████████████████████████                                                        | 2/6 [02:13<04:28, 67.25s/it]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▊                                                                   | 1/5 [00:14<00:58, 14.64s/it]

 40%|███████████████████████████

 20%|████████████████▊                                                                   | 1/5 [00:13<00:52, 13.01s/it]

 40%|█████████████████████████████████▌                                                  | 2/5 [00:25<00:38, 12.91s/it]

 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:38<00:25, 12.94s/it]

 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:51<00:12, 12.98s/it]

 83%|██████████████████████████████████████████████████████████████████████              | 5/6 [05:38<01:06, 66.99s/it]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

 20%|████████████████▊                                                                   | 1/5 [00:12<00:51, 12.93s/it]

 40%|█████████████████████████████████▌                                                  | 2/5 [00:25<00:38, 12.85s/it]

 60%|███████████████████████████

In [91]:
res = pd.DataFrame(model_results)

In [93]:
res.sort_values(by = "Coherence", ascending= False)

,Topics,Alpha,Beta,Coherence
178,7,asymmetric,0.91,0.396868
148,6,asymmetric,0.91,0.375354
193,8,0.61,0.91,0.364687
198,8,0.91,0.91,0.357554
163,7,0.61,0.91,0.357351
...,...,...,...,...
25,2,asymmetric,0.01,0.251559
28,2,asymmetric,0.91,0.248604
16,2,0.91,0.31,0.246131
10,2,0.61,0.01,0.242793


In [94]:
lda_model = gensim.models.LdaMulticore(corpus=doc_term_matrix,
                                           id2word=dictionary,
                                           num_topics=7, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha='asymmetric',
                                           eta=0.91)

In [95]:
lda_model.save('./data/best_description_model')